In [1]:
import plotly.express as px
from dash import Dash, dcc, html, Output, Input

In [6]:
import wget
import pandas as pd 
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file) 



In [12]:
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [24]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Assuming spacex_df is already loaded
app = Dash(__name__)

# Layout of the application
app.layout = html.Div(
    children=[
        html.H1(
            "Launch Site Success Rate",
            style={"fontSize": "40px", "textAlign": "center", "color": "blue"},
        ),
        html.P(
            "Success rate of each Launch Site",
            style={"textAlign": "center", "color": "blue", "fontSize": "30px"},
        ),
        dcc.Dropdown(
            id="site-dropdown",
            options=[
                {"label": x, "value": x} for x in spacex_df["Launch Site"].unique()
            ] + [{"label": "All Sites", "value": "all"}],
            value="all",
            placeholder="Select a Launch Site",
        ),
        dcc.Graph(id="launch-site-chart"),
    ]
)

# Callback to update the chart based on dropdown selection
@app.callback(Output("launch-site-chart", "figure"), Input("site-dropdown", "value"))
def dash_chart(selected_site):
    if selected_site == "all":
        dff = spacex_df
        title = "Launch Site Success Rate for All Sites"
    else:
        dff = spacex_df[spacex_df["Launch Site"] == selected_site]
        title = f"Launch Site Success Rate for {selected_site}"

    # Create the chart using plotly express
    fig = px.pie(
        dff,
        names="class",
        title=title,
        hole=0.3,
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)


In [30]:
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd

# Assume spacex_df is preloaded
# Example dataset columns: 'Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version Category'
min_payload = spacex_df['Payload Mass (kg)'].min()
max_payload = spacex_df['Payload Mass (kg)'].max()

# Initialize the Dash app
app = Dash(__name__)

# Layout of the application
app.layout = html.Div(
    children=[
        html.H1(
            "SpaceX Launch Records Dashboard",
            style={"textAlign": "center", "color": "#503D36", "fontSize": 40},
        ),
        html.P(
            "Analyze SpaceX Launch Outcomes",
            style={"textAlign": "center", "color": "#503D36", "fontSize": 30},
        ),
        dcc.Dropdown(
            id="site-dropdown",
            options=[
                {"label": site, "value": site} for site in spacex_df["Launch Site"].unique()
            ] + [{"label": "All Sites", "value": "all"}],
            value="all",
            placeholder="Select a Launch Site",
            searchable=True,
        ),
        html.Br(),
        dcc.Graph(id="success-pie-chart"),
        html.Br(),
        html.P("Payload range (Kg):"),
        dcc.RangeSlider(
            id="payload-slider",
            min=0,
            max=10000,
            step=1000,
            value=[min_payload, max_payload],
        ),
        html.Br(),
        dcc.Graph(id="success-payload-scatter-chart"),
    ]
)

# Callback for pie chart
@app.callback(
    Output(component_id="success-pie-chart", component_property="figure"),
    Input(component_id="site-dropdown", component_property="value"),
)
def update_pie_chart(selected_site):
    if selected_site == "all":
        # Total success and failure counts across all sites
        success_counts = spacex_df["class"].value_counts().reset_index()
        fig = px.pie(
            success_counts,
            names="index",
            values="class",
            title="Total Success Launches for All Sites",
        )
    else:
        # Success and failure counts for the selected site
        filtered_df = spacex_df[spacex_df["Launch Site"] == selected_site]
        success_counts = filtered_df["class"].value_counts().reset_index()
        fig = px.pie(
            success_counts,
            names="index",
            values="class",
            title=f"Total Success Launches for {selected_site}",
        )
    return fig

# Callback for scatter plot
@app.callback(
    Output(component_id="success-payload-scatter-chart", component_property="figure"),
    [
        Input(component_id="site-dropdown", component_property="value"),
        Input(component_id="payload-slider", component_property="value"),
    ],
)
def update_scatter_plot(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[
        (spacex_df["Payload Mass (kg)"] >= low) & (spacex_df["Payload Mass (kg)"] <= high)
    ]

    if selected_site == "all":
        fig = px.scatter(
            filtered_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title="Correlation Between Payload and Success for All Sites",
        )
    else:
        filtered_df = filtered_df[filtered_df["Launch Site"] == selected_site]
        fig = px.scatter(
            filtered_df,
            x="Payload Mass (kg)",
            y="class",
            color="Booster Version Category",
            title=f"Correlation Between Payload and Success for {selected_site}",
        )
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True, port=8052)
